In [ ]:
import pandas as pd
import numpy as np
import operator
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Loads in the different .feather datasets


In [ ]:
anp_df = pd.read_feather('data/anp.feather')
image_d_df = pd.read_feather('data/image_data.feather')
image_m_df = pd.read_feather('data/image_metrics.feather')
face_df = pd.read_feather('data/face.feather')
object_df = pd.read_feather('data/object_labels.feather')
survey_df = pd.read_feather('data/survey.feather')

In [ ]:
image_d_df.to_csv('image_d_df.csv')
image_m_df.to_csv('image_m_df.csv')


In [ ]:
image_d_df = pd.read_feather('data/image_data.feather')

# Adding together image data, anp, object label and face.

The next step is to create one dataframe, grouped per user, containing the aggregate data that was pulled from the users their instagram accounts. 


## anp_df

#### method:

1. the anp label is dropped because it does not make sense to aggregate this. 
2. the dataframe is then split into a sentiment dataframe and an emotion dataframe for seperate processing.

In [ ]:
# drop anp_label

anp_df = anp_df.drop(['anp_label'], axis=1)

In [ ]:
# make sentiment df

anp_sent = anp_df.drop(['emotion_score', 'emotion_label'], axis = 1)
anp_sent = pd.DataFrame(anp_sent.groupby(by='image_id', as_index=False)['anp_sentiment'].mean())
anp_sent = anp_sent.set_index('image_id')

In [ ]:
# split for emotion

anp_emo = anp_df.drop(['anp_sentiment'], axis = 1)
len(anp_emo['image_id'].unique())

In [ ]:
# the emotion with the highest score is taken for each user

ax = anp_emo.groupby(['image_id'])['emotion_score'].transform(max) == anp_emo['emotion_score']
anp_emo = anp_emo[ax]

In [ ]:
# duplicates are removed to get only unique images

anp_emo = anp_emo.drop_duplicates()
anp_emo = anp_emo.set_index('image_id')


In [ ]:
# the emotion and sentiment dataframes are merged again.

anp_final = anp_sent.join(anp_emo)

## Image_d_df

#### method:
1. Thing that can not be aggregated such as image_link, etc, are dropped from the dataframe. 
2. Timestamp is changed so that only the hour from 00 - 23 is included.

In [ ]:
# drop unnecesarry data

image_d_df = image_d_df.drop(['image_link','image_url', 'user_full_name', 'user_name'\
                              , 'user_website','user_profile_pic', 'user_bio', 'image_posted_time_unix'], axis=1)



In [ ]:
# change timestamp to be 24 hours

import re

def changeTime(time):
    time = time.split()[1]
    time = time.split(':')[0]
    time = time.lstrip('0')
    time = ''.join([i for i in time if re.match('[0-9]', i) is not None])
    try: 
        return int(time)
    except:
        return 0

image_d_df['posting_time'] = image_d_df['image_posted_time'].apply(lambda x: changeTime(x))

In [ ]:
# set indesx to image id and drop image_posted_time

image_d_df = image_d_df.set_index('image_id')
image_d_df = image_d_df.drop('image_posted_time', axis=1)

## Image_m_df

#### method: 
1. drop things that we do not need

In [ ]:
image_m_df = image_m_df.drop(['comment_count_time_created', 'like_count_time_created'], axis=1)


In [ ]:
image_m_df = image_m_df.set_index('image_id')
image_m_df



## object_df

#### method:
1. Only take the object with highest confidence level for each picture. 


In [ ]:
# take object with max confidence

group = object_df.groupby(['image_id', 'data_amz_label'])['data_amz_label_confidence'].transform(max) == object_df['data_amz_label_confidence']
object_df = object_df[group]

## face_df: emotion

#### method:

1. first a split is made for face emotion
2. images are grouped by image_id and face_id, for each image we check how many faces have a certain emotion. The highest number of face emotions is picked as the image emotion.
3. If we have an equal number (let's say 1 joy face and 1 sad face, the emotion with the highest confidence level is selected.) 


In [ ]:
# first, split for face emotion

face_emo = face_df[['image_id', 'face_id', 'face_emo', 'emo_confidence']]
ab = face_emo.groupby(['image_id', 'face_id'])['emo_confidence'].transform(max) == face_emo['emo_confidence']
face_emo = face_emo[ab]


In [ ]:

# get unique image_id's
UniqueNames = face_emo.image_id.unique()

def changeDataFrame(df):
    
    new_list = []
    
    for _id in UniqueNames:
        
        # check whether one emotion is found more often than others
        
        df_temp = face_emo.loc[face_emo['image_id'] == _id]
        counts = df_temp['face_emo'].value_counts().to_dict()
        maximum = sorted(counts.items(), key=operator.itemgetter(1))

        # if there is an equal number of emotions look at confidence.
        
        if len(maximum) > 1:
            if maximum[0][1] == maximum[1][1]:

                group = df_temp.groupby(['image_id'])['emo_confidence'].transform(max) == df_temp['emo_confidence']
                new_slice = df_temp[group]
                new_list.append([new_slice['image_id'].values[0], new_slice['face_emo'].values[0]])  
                
            else: 
                new_list.append([df_temp['image_id'].values[0], maximum[0][0]])

        # else select the emotion that is present on most faces.
        
        else: 
            new_list.append([df_temp['image_id'].values[0], maximum[0][0]])
            
    output_df = pd.DataFrame(new_list, columns=['image_id','face_emo'])
    
    return output_df
    

new_emo = changeDataFrame(face_emo)

## face_df: finding the number of faces

#### method:
1. aggregate number of faces per picture


In [ ]:
# take image_id, face_id and emo_confidence

face_no_df = face_df[['image_id', 'face_id', 'emo_confidence']]
group = face_no_df.groupby(['image_id', 'face_id'])['emo_confidence'].transform(max) == face_no_df['emo_confidence']
face_no_df = face_no_df[group].drop('emo_confidence', axis=1)

In [ ]:
# aggregate for number of faces 
face_count_df = pd.DataFrame(face_no_df['image_id'].value_counts())
face_count_df['no_of_faces'] = face_count_df['image_id']
face_count_df = face_count_df.drop('image_id', axis=1)


## face_df: find average age per picture

1. select face age low and high
2. take the mean value for each face
3. take the mean of those values for each picture

In [ ]:
face_df_age = face_df[['image_id', 'face_age_range_low', 'face_age_range_high']]

In [ ]:
# taking mean per face
face_df_age['average_age'] = (face_df_age['face_age_range_low'] + face_df_age['face_age_range_high'])/2
face_df_age

# taking mean per picture
average_age_df = pd.DataFrame(face_df_age.groupby('image_id')['average_age'].mean())

In [ ]:
# object_df.to_csv('object.csv')
# face_df.to_csv('face.csv')

In [ ]:
# survey_df.to_csv('survey.csv')

In [ ]:
image_d_df

# Merge different dataframes

1. join different dataframes by image_id
2. aggregate for user_id

In [ ]:
# merge different dataframes
df1 = anp_final.join(image_d_df)
df2 = df1.join(face_count_df)
df3 = df2.join(average_age_df)
df4 = df3.join(image_m_df)



'''
' TODO
' Also join object dataframe and new_emo dataframe
'''

In [ ]:
df4

In [ ]:
quant_df = df4.drop(['emotion_label', 'emotion_score', 'image_filter'], axis=1)
quant_df

In [ ]:
quant_df = quant_df.groupby(by='user_id').mean()
# quant_df.to_csv('analysisfile.csv')
quant_df

In [ ]:
# quant_df.to_csv('analysis.csv')

## Extra: try to get smile percentage per picture

In [ ]:
# Takes all the face smiles where the confidence level is more than 80 percent
face_smile_df = face_df.loc[face_df['face_smile_confidence'] > 80]

# make dataframe split to include smile
face_smile_df = face_smile_df[['image_id', 'face_smile', 'face_id']]

# drop duplicate values
face_smile_df = face_smile_df.drop_duplicates()
face_smile_df = face_smile_df.drop('face_id', axis=1)

group = face_smile_df.groupby(['image_id', 'face_smile'])['face_smile'].count()
group = pd.DataFrame(group / group.groupby(level=0).sum())
# face_smile_df[group]

group
# face_smile_df = group.loc[group['face_smile'] == True]
# face_smile_df


# face_smile_df = face_smile_df.groupby(by='image_id')['face_smile'].count()
# face_smile_df.head

## Merge survey dataframe and analysis dataframe

In [ ]:
survey_df = pd.read_feather('data/survey.feather')

In [ ]:
survey_df['insta_user_id'] = survey_df['insta_user_id'].apply(lambda x: str(int(x)))

In [ ]:
survey_df = survey_df.set_index('insta_user_id')


In [ ]:
survey_df

In [ ]:
regr_df = quant_df.join(survey_df)

In [ ]:

regr_df = regr_df.drop(['A_2', 'N_1', 'P_1', 'E_1', 'A_1',
       'H_1', 'M_1', 'R_1', 'M_2', 'E_2', 'LON', 'H_2', 'P_2', 'N_2', 'A_3',
       'N_3', 'E_3', 'H_3', 'R_2', 'M_3', 'R_3', 'P_3', 'HAP','network_id', 'P', 'E', 'R', 'M', 'A', 'start_q', 'end_q', 'private_account'], axis=1)



In [ ]:
image_d_df['emotion)']

In [ ]:
regr_df

In [ ]:
from pandas.plotting import scatter_matrix
%matplotlib inline

scatter_matrix(regr_df[['PERMA','anp_sentiment', 'data_memorability',
       'user_followed_by', 'user_follows', 'user_posted_photos',
       'posting_time', 'no_of_faces', 'average_age','imagecount']], alpha=1.0, figsize=(20, 20), diagonal='kde')


In [ ]:
regr_df.columns

In [ ]:
# Take two variables
X = regr_df[["user_posted_photos"]]
y = regr_df[["PERMA"]]

# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# This creates a LinearRegression object
lm = LinearRegression()

# Fit curve
lm.fit(X_train, y_train)

# Make prediction
y_predict = lm.predict(X_test)

# Show metrics
print('Coefficients: \n', lm.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_predict))



# results from regression

#### 1. PERMA ~ anp_sentiment

Correlation: 0.21477

Coefficients: 
 [[ 4.68761213]]
Mean squared error: 3.59
Variance score: 0.03

#### 2. PERMA ~ posting_time

correlation: -0.205047

Coefficients: 
 [[-0.12492383]]
Mean squared error: 2.93
Variance score: -0.08

#### 3. PERMA ~ data_memorability

correlation: 0.195383

Coefficients: 
 [[ 9.01465037]]
Mean squared error: 1.98
Variance score: -0.04

#### 4. PERMA ~ imagecount	

correlation: -0.112792

Coefficients: 
 [[-0.00031376]]
Mean squared error: 3.32
Variance score: 0.04
